# NLP 2025-26 -- LLM Project: Email Routing Agents

**Objective:** Develop three LLM-based agents to automatically route customer support emails to the appropriate department.

**Departments:** Technical Support, Customer Service, Billing and Payments, Sales and Pre-Sales, General Inquiry

**Agents:**
1. Routing with prompting using GPT-2 (frozen model)
2. Routing with fine-tuning (LoRA) on GPT-2
3. Routing with discriminative classifier using DistilBERT

## 0. Setup and Imports

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import time
import tracemalloc
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from transformers import (
    GPT2Tokenizer, GPT2LMHeadModel,
    DistilBertTokenizer, DistilBertForSequenceClassification,
)
from tqdm import tqdm

from datapreparation import load_and_prepare_data

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

## 1. Data Loading and Exploration

In [ ]:
train_ds, val_ds, test_ds, label_list, label2id, id2label = load_and_prepare_data()

print(f"Train size: {len(train_ds)}")
print(f"Validation size: {len(val_ds)}")
print(f"Test size: {len(test_ds)}")
print(f"Labels: {label_list}")

In [ ]:
# Visualize label distribution
from collections import Counter

fig, axes = plt.subplots(1, 3, figsize=(18, 5))
for ax, (name, split) in zip(axes, [("Train", train_ds), ("Validation", val_ds), ("Test", test_ds)]):
    counts = Counter(split["queue"])
    ax.bar(range(len(counts)), counts.values())
    ax.set_xticks(range(len(counts)))
    ax.set_xticklabels(counts.keys(), rotation=45, ha="right")
    ax.set_title(f"{name} Set Distribution")
    ax.set_ylabel("Count")
plt.tight_layout()
plt.show()

In [ ]:
# Inspect a few examples
for i in range(3):
    print(f"--- Example {i+1} ---")
    print(f"Subject: {train_ds[i]['subject']}")
    print(f"Body: {train_ds[i]['body'][:200]}...")
    print(f"Department: {train_ds[i]['queue']}")
    print()

---
## 2. Agent 1: Routing with Prompting (Frozen GPT-2)

Use a pretrained GPT-2 model without updating weights. The model receives an instruction-style prompt and must output the department name.

*Implementation coming in feature/agent1-prompting*

---
## 3. Agent 2: Routing with Fine-Tuning (LoRA on GPT-2)

Fine-tune GPT-2 using LoRA, then evaluate with the same prompt as Agent 1.

*Implementation coming in feature/agent2-finetuning*

---
## 4. Agent 3: Routing with DistilBERT Classifier

Fine-tune DistilBERT for sequence classification over the 5 department labels.

*Implementation coming in feature/agent3-classifier*

---
## 5. Results Comparison

Compare all agents on the test set in terms of accuracy, computational time, and memory usage.

*Implementation coming in feature/comparison*